# Dask Array

Материалы:
* Макрушин С.В. Лекция 11: Dask
* https://docs.dask.org/en/latest/array.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

In [46]:
import dask.array as da
import dask.dataframe as dd
import numpy as np
import h5py

C:\Users\Lisa\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

In [2]:
arr_np = np.random.normal(0, 1, size=(1000, 300000))
arr_np

array([[ 1.70219437e+00,  1.86629082e-01,  1.00908016e+00, ...,
        -1.23057499e+00, -1.75280182e-01,  2.04955011e+00],
       [ 1.19427595e+00, -6.01007426e-01,  3.74273349e-01, ...,
         1.82200659e+00,  1.49019543e+00, -1.29549271e+00],
       [-6.60943217e-01,  4.46440680e-01, -1.50960948e+00, ...,
        -4.21759167e-01, -1.16422037e+00, -2.46781645e+00],
       ...,
       [-6.73994461e-02,  8.69588877e-02,  1.16015757e+00, ...,
        -1.91507600e+00, -3.82063559e-01, -7.87731229e-01],
       [ 3.84454070e-01,  1.46239366e-01,  1.60310174e-03, ...,
         7.31004991e-01, -1.00926844e+00,  8.38220885e-01],
       [ 5.81995683e-01, -8.94245855e-02,  1.74636036e-01, ...,
         1.72728279e-01,  2.97838179e-01,  8.62934576e-01]])

In [3]:
arr_da = da.random.normal(0, 1, size=(1000, 300000), chunks=(1000, 300000))
arr_da

C:\ProgramData\Anaconda3\lib\site-packages\dask\array\core.py:1182: RuntimeWarning: overflow encountered in long_scalars
  cbytes = format_bytes(np.prod(self.chunksize) * self.dtype.itemsize)


dask.array<normal, shape=(1000, 300000), dtype=float64, chunksize=(1000, 300000)>

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [4]:
%%time
np.power(arr_np, 2).sum()

Wall time: 1min 2s


300012834.9966135

In [5]:
%%time
da.power(arr_da, 2).sum().compute()

Wall time: 53.6 s


300038812.50520015

3. Визуализируйте граф вычислений для задачи 2.

## Лабораторная работа 11

1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [2]:
hdf = h5py.File('minutes_n_ingredients_full.hdf5', 'r')
recipe = da.from_array(hdf['recipe'], chunks=(100000, 3))

In [3]:
recipe

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [4]:
recipe[:, 1:].mean(axis=0).compute()

array([1004.20805176,    5.4198008 ])

3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

In [5]:
%%time
recipe[:, 1:].mean(axis=0).compute()

Wall time: 137 ms


array([1004.20805176,    5.4198008 ])

In [6]:
%%time
recipe1 = da.from_array(hdf['recipe'], chunks=(1000, 3))
recipe1[:, 1:].mean(axis=0).compute()

Wall time: 1.94 s


array([1004.20805176,    5.4198008 ])

4. Выберите рецепты, время выполнения которых меньше медианного значения

In [11]:
da.median(recipe[:, 1], axis=0).compute()

32.0

In [12]:
recipe[recipe[:, 1] < da.median(recipe[:, 1], axis=0)].compute()

array([[1089012,      23,       5],
       [1428572,       0,       5],
       [1400250,      24,       1],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]], dtype=int64)

5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [15]:
cnt = da.bincount(recipe[:, 2]).compute()
i = np.nonzero(cnt)[0]
da.vstack((i, cnt[i])).T.compute()

array([[     1, 222071],
       [     2, 224158],
       [     3, 229388],
       [     4, 234948],
       [     5, 240720],
       [     6, 244360],
       [     7, 247181],
       [     8, 246747],
       [     9, 246816],
       [    10,  22430],
       [    11,  19094],
       [    12,  15165],
       [    13,  11640],
       [    14,   8284],
       [    15,   6014],
       [    16,   4145],
       [    17,   2793],
       [    18,   1913],
       [    19,   1279],
       [    20,    852],
       [    21,    529],
       [    22,    346],
       [    23,    244],
       [    24,    178],
       [    25,    107],
       [    26,     68],
       [    27,     55],
       [    28,     33],
       [    29,     22],
       [    30,     20],
       [    31,     13],
       [    32,      5],
       [    33,      4],
       [    34,      3],
       [    35,      4],
       [    36,      1],
       [    37,      2],
       [    38,      1],
       [    39,      1],
       [    40,      2],


6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [16]:
max_time = recipe[recipe[:, 1] == recipe[:, 1].max(axis=0), 1].compute()
max_time

array([2147483647], dtype=int64)

In [44]:
quan = da.percentile(recipe[:, 1], q =75).compute()
quan

array([49.])

In [50]:
recipe[recipe[:, 1] > quan, 1].compute() 

array([60, 57, 62, ..., 61, 60, 64], dtype=int64)

In [54]:
a = recipe[0, 1].compute() 

In [57]:
a = quan
a

array([49.])

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

In [43]:
pref = da.array([15, 6])
recipe[(abs(da.linalg.norm(recipe[:, 1:], 1, axis=1) - da.linalg.norm(pref, 1))).argmin()].compute() 

array([1132618,      15,       6], dtype=int64)

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом;
2. агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных.

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент